In [1]:
## Basic stuff
%load_ext autoreload
%autoreload
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>div.output_area{max-height:10000px;overflow:scroll;}</style>"""))

## Python Version
import sys
print("Python: {0}".format(sys.version))

from ioUtils import saveJoblib, loadJoblib, saveFile, getFile
import urllib
from urllib.parse import quote

from discogsBase import discogs
from discogsUtils import discogsUtils
from collection import collections
from artist import artist
from timeUtils import clock, elapsed
from fsUtils import moveFile, setFile, setDir, setSubDir, isFile, isDir, mkDir
from fileUtils import getFileBasics, getBasename
from artists import artists
from albums import albums
from time import sleep
import time
from multiprocessing import Pool

import datetime as dt
start = dt.datetime.now()
print("Notebook Last Run Initiated: "+str(start))

Python: 3.7.3 (default, Mar 27 2019, 16:54:48) 
[Clang 4.0.1 (tags/RELEASE_401/final)]
Notebook Last Run Initiated: 2020-01-17 18:41:57.339224


In [2]:
%load_ext autoreload
%autoreload
disc = discogs()
arts = artists(disc)
albs = albums(disc)
dutils = discogsUtils()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Saved Discog Directory /Volumes/Music/Discog is Available
Local Discog Directory /Users/tgadfort/Music/Discog is Available
/Volumes/Music/Discog/collections exists
/Volumes/Music/Discog/artists exists
/Volumes/Music/Discog/albums exists
/Volumes/Music/Discog/collections-db exists
/Volumes/Music/Discog/artists-db exists
/Volumes/Music/Discog/albums-db exists
/Volumes/Music/Discog/artists-db/metadata exists
/Volumes/Music/Discog/albums-db/metadata exists
/Volumes/Music/Discog/diagnostic exists
/Volumes/Music/Discog/db exists


# Collections

In [ ]:
%load_ext autoreload
%autoreload
#col = collections(disc)
#col.parseCollections(force=False)
#col.createCollectionDBs()

# Parse Artists (Parallel)

In [18]:
%load_ext autoreload
%autoreload

from artist import artist
from artists import artists
arts = artists(disc)

def parseArtists(modVal, force=False, doExtra=False):
    arts.parseArtistModValFiles(modVal, force=force)
    #arts.createArtistModValMetadata(modVal=modVal)
    #arts.createArtistAlbumModValMetadata(modVal=modVal)
    
def parseArtistsParallel(nProcs=3, force=False):
    pool = Pool(processes=nProcs)
    #result = pool.map_async(parseArtists, range(100)) 
    result = pool.map_async(parseArtists, [97, 96, 98, 62])

    while not result.ready():
        if force is True:
            time.sleep(10)
        else:
            time.sleep(1)
    print("")
    return result.get()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
#arts.parseArtistModValFiles(27, force=True)
#arts.parseArtistModValFiles(50, force=True)

In [20]:
start,cmt=clock("Parsing All Artists")
start2,cmt2=clock("Parallel Artist Parsing")
parseArtistsParallel()
elapsed(start2, cmt2)
elapsed(start, cmt)

Current Time is Wed Jan 22, 2020 18:17:37 for Parsing All Artists
Current Time is Wed Jan 22, 2020 18:17:37 for Parallel Artist Parsing
Parsing Artist Files For ModVal 97
Parsing Artist Files For ModVal 96
Parsing Artist Files For ModVal 98
  ===> Found 7074 files. There are 419 new files to parse.
  ===> Found 7077 files. There are 438 new files to parse.
  ===> Found 7100 files. There are 444 new files to parse.
Saving 444 new artist IDs to /Volumes/Music/Discog/artists-db/97-DB.p
Saving 438 new artist IDs to /Volumes/Music/Discog/artists-db/96-DB.p
Saving 420 new artist IDs to /Volumes/Music/Discog/artists-db/98-DB.p
  --> This file is 11.2MB.
Saving 7100 new artist IDs name data to /Volumes/Music/Discog/artists-db/metadata/97-Metadata.p
  --> This file is 411.2kB.
Saving 7100 new artist IDs media data to /Volumes/Music/Discog/artists-db/metadata/97-MediaMetadata.p
  --> This file is 3.6MB.
Parsing Artist Files For ModVal 62
  --> This file is 12.3MB.
Saving 7077 new artist IDs name

# Albums

In [5]:
%load_ext autoreload
%autoreload
from album import album
from albums import albums
from discogsBase import discogs

disc = discogs()
albs = albums(disc)
#albs.parseAlbumModValFiles(modVal=27)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Saved Discog Directory /Volumes/Music/Discog is Available
Local Discog Directory /Users/tgadfort/Music/Discog is Available
/Volumes/Music/Discog/collections exists
/Volumes/Music/Discog/artists exists
/Volumes/Music/Discog/albums exists
/Volumes/Music/Discog/collections-db exists
/Volumes/Music/Discog/artists-db exists
/Volumes/Music/Discog/albums-db exists
/Volumes/Music/Discog/artists-db/metadata exists
/Volumes/Music/Discog/albums-db/metadata exists
/Volumes/Music/Discog/diagnostic exists
/Volumes/Music/Discog/db exists


In [21]:
def parseAlbums(modVal):
    albs.parseAlbumModValFiles(modVal=modVal)
    #albs.createAlbumModValMetadata(modVal)
    #albs.createAlbumModValMetadata(modVal=modVal)

def parseAlbumsParallel(modVals=None):
    if modVals is None:
        modVals = ["NAN"] + list(range(100))
        #modVals = range(100)
    nProcs = 2
    pool = Pool(processes=nProcs)
    result = pool.map_async(parseAlbums, modVals)
 
    start, cmt = clock("Processing {0} modVals using {1} processes".format(modVals, nProcs))
    print("Runing .",end="")
    while not result.ready():
        #print(".",end="")
        time.sleep(10)
    print("")
    
    elapsed(start, cmt)
    return result.get()

In [22]:
%load_ext autoreload
%autoreload
from album import album
from albums import albums
albs = albums(disc)

start,cmt=clock("Parsing All Albums")
start2,cmt2=clock("Parallel Album Parsing")
parseAlbumsParallel()
elapsed(start2, cmt2)
elapsed(start, cmt)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Current Time is Wed Jan 22, 2020 18:27:15 for Parsing All Albums
Current Time is Wed Jan 22, 2020 18:27:15 for Parallel Album Parsing
Current Time is Wed Jan 22, 2020 18:27:15 for Parsing Album Files For ModVal NAN
Current Time is Wed Jan 22, 2020 18:27:15 for Parsing Album Files For ModVal 12
Current Time is Wed Jan 22, 2020 18:27:15 for Processing ['NAN', 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99] modVals using 2 processes
Runing .  Looking for artist albums in /Volumes/Music/Discog/albums/NAN
  Looking for artist albums in /Volumes/Music/Discog/albums/12
 

  --> This file is 2.6MB.
Current Time is Wed Jan 22, 2020 18:37:30 for Done with Parsing Album Files For ModVal 15
Process [{0}] took 2.4 minutes.
Current Time is Wed Jan 22, 2020 18:37:30 for Parsing Album Files For ModVal 16
  Looking for artist albums in /Volumes/Music/Discog/albums/16
  Found 5272 artist directories
    Found 0   albums (0    ) in 0/5272	/Volumes/Music/Discog/albums/16/1000016
    Found 0   albums (0    ) in 100/5272	/Volumes/Music/Discog/albums/16/1082316
    Found 0   albums (1    ) in 500/5272	/Volumes/Music/Discog/albums/16/1360916
    Found 0   albums (1    ) in 1000/5272	/Volumes/Music/Discog/albums/16/170716
    Found 0   albums (5    ) in 1500/5272	/Volumes/Music/Discog/albums/16/2042916
    Found 0   albums (7    ) in 2000/5272	/Volumes/Music/Discog/albums/16/2436616
    Found 0   albums (8    ) in 2500/5272	/Volumes/Music/Discog/albums/16/280616
    Found 0   albums (10   ) in 3000/5272	/Volumes/Music/Discog/albums/16/333516
    Found 0   albums (12   ) 

    Found 0   albums (19   ) in 2500/4710	/Volumes/Music/Discog/albums/1/3523901
    Found 0   albums (25   ) in 3000/4710	/Volumes/Music/Discog/albums/1/4232101
    Found 0   albums (27   ) in 3500/4710	/Volumes/Music/Discog/albums/1/5013801
    Found 0   albums (29   ) in 4000/4710	/Volumes/Music/Discog/albums/1/601701
  Found 6362 artist directories
    Found 0   albums (0    ) in 0/6362	/Volumes/Music/Discog/albums/19/1000119
    Found 0   albums (0    ) in 100/6362	/Volumes/Music/Discog/albums/19/1061019
    Found 0   albums (34   ) in 4500/4710	/Volumes/Music/Discog/albums/1/846301
    Found 0   albums (35   ) in 4709/4710	/Volumes/Music/Discog/albums/1/997001
Saving 35 (new) / 35 (parsed) / 20114 (total) album IDs to /Volumes/Music/Discog/albums-db/1-DB.p
    Found 0   albums (2    ) in 500/6362	/Volumes/Music/Discog/albums/19/1361919
    Found 0   albums (5    ) in 1000/6362	/Volumes/Music/Discog/albums/19/1712519
    Found 0   albums (6    ) in 1500/6362	/Volumes/Music/Discog/

Saving 26 (new) / 26 (parsed) / 12603 (total) album IDs to /Volumes/Music/Discog/albums-db/21-DB.p
  --> This file is 13.8MB.
Saving 6887 new artist IDs to /Volumes/Music/Discog/albums-db/metadata/3-ArtistMetadata.p
  --> This file is 1.2MB.
Saving 6887 new artist IDs to /Volumes/Music/Discog/albums-db/metadata/3-ArtistAlbums.p
  --> This file is 2.1MB.
Current Time is Wed Jan 22, 2020 18:49:04 for Done with Parsing Album Files For ModVal 3
Process [{0}] took 2.0 minutes.
Current Time is Wed Jan 22, 2020 18:49:05 for Parsing Album Files For ModVal 4
  Looking for artist albums in /Volumes/Music/Discog/albums/4
  --> This file is 9.5MB.
Saving 3822 new artist IDs to /Volumes/Music/Discog/albums-db/metadata/21-ArtistMetadata.p
  --> This file is 828.5kB.
Saving 3822 new artist IDs to /Volumes/Music/Discog/albums-db/metadata/21-ArtistAlbums.p
  --> This file is 1.4MB.
Current Time is Wed Jan 22, 2020 18:49:09 for Done with Parsing Album Files For ModVal 21
Process [{0}] took 1.6 minutes.


    Found 0   albums (5    ) in 3000/6614	/Volumes/Music/Discog/albums/6/3149506
    Found 0   albums (7    ) in 3500/6614	/Volumes/Music/Discog/albums/6/3535206
    Found 0   albums (7    ) in 4000/6614	/Volumes/Music/Discog/albums/6/3936906
    Found 0   albums (7    ) in 4500/6614	/Volumes/Music/Discog/albums/6/452106
    Found 0   albums (7    ) in 5000/6614	/Volumes/Music/Discog/albums/6/5056006
    Found 0   albums (7    ) in 5500/6614	/Volumes/Music/Discog/albums/6/558706
    Found 0   albums (11   ) in 6000/6614	/Volumes/Music/Discog/albums/6/6884606
    Found 0   albums (14   ) in 6500/6614	/Volumes/Music/Discog/albums/6/931406
  --> This file is 19.6MB.
    Found 0   albums (15   ) in 6613/6614	/Volumes/Music/Discog/albums/6/999806
Saving 15 (new) / 15 (parsed) / 42867 (total) album IDs to /Volumes/Music/Discog/albums-db/6-DB.p
Saving 6070 new artist IDs to /Volumes/Music/Discog/albums-db/metadata/23-ArtistMetadata.p
  --> This file is 1.3MB.
Saving 6070 new artist IDs to /Vo

...

Process [{0}] took 2.4 minutes.
Current Time is Wed Jan 22, 2020 20:10:31 for Parsing Album Files For ModVal 76
  Looking for artist albums in /Volumes/Music/Discog/albums/76
  Found 6725 artist directories
    Found 0   albums (0    ) in 0/6725	/Volumes/Music/Discog/albums/76/1000176
    Found 0   albums (0    ) in 100/6725	/Volumes/Music/Discog/albums/76/1057076
    Found 0   albums (2    ) in 500/6725	/Volumes/Music/Discog/albums/76/1329776
    Found 0   albums (3    ) in 1000/6725	/Volumes/Music/Discog/albums/76/164276
    Found 0   albums (7    ) in 1500/6725	/Volumes/Music/Discog/albums/76/1943976
    Found 0   albums (9    ) in 2000/6725	/Volumes/Music/Discog/albums/76/2293176
    Found 0   albums (10   ) in 2500/6725	/Volumes/Music/Discog/albums/76/2639676
  --> This file is 21.7MB.
Saving 6446 new artist IDs to /Volumes/Music/Discog/albums-db/metadata/78-ArtistMetadata.p
  --> This file is 1.3MB.
Saving 6446 new artist IDs to /Volumes/Music/Discog/albums-db/metadata/78-ArtistA

    Found 0   albums (19   ) in 5500/6347	/Volumes/Music/Discog/albums/80/6092080
    Found 0   albums (21   ) in 4500/4602	/Volumes/Music/Discog/albums/91/918791
    Found 0   albums (23   ) in 4601/4602	/Volumes/Music/Discog/albums/91/999491
Saving 23 (new) / 23 (parsed) / 16762 (total) album IDs to /Volumes/Music/Discog/albums-db/91-DB.p
    Found 0   albums (20   ) in 6000/6347	/Volumes/Music/Discog/albums/80/792780
    Found 0   albums (22   ) in 6346/6347	/Volumes/Music/Discog/albums/80/998980
Saving 22 (new) / 22 (parsed) / 31134 (total) album IDs to /Volumes/Music/Discog/albums-db/80-DB.p
  --> This file is 12.5MB.
Saving 4602 new artist IDs to /Volumes/Music/Discog/albums-db/metadata/91-ArtistMetadata.p
  --> This file is 977.7kB.
Saving 4602 new artist IDs to /Volumes/Music/Discog/albums-db/metadata/91-ArtistAlbums.p
  --> This file is 1.8MB.
Current Time is Wed Jan 22, 2020 20:17:00 for Done with Parsing Album Files For ModVal 91
Process [{0}] took 2.0 minutes.
Current Time 

    Found 0   albums (23   ) in 6780/6781	/Volumes/Music/Discog/albums/82/999982
Saving 23 (new) / 23 (parsed) / 21016 (total) album IDs to /Volumes/Music/Discog/albums-db/82-DB.p
  Found 6354 artist directories
    Found 0   albums (0    ) in 0/6354	/Volumes/Music/Discog/albums/94/1001094
    Found 0   albums (0    ) in 100/6354	/Volumes/Music/Discog/albums/94/1062294
  --> This file is 14.2MB.
Saving 6781 new artist IDs to /Volumes/Music/Discog/albums-db/metadata/82-ArtistMetadata.p
  --> This file is 1.2MB.
Saving 6781 new artist IDs to /Volumes/Music/Discog/albums-db/metadata/82-ArtistAlbums.p
    Found 0   albums (1    ) in 500/6354	/Volumes/Music/Discog/albums/94/1358994
    Found 0   albums (2    ) in 1000/6354	/Volumes/Music/Discog/albums/94/1661294
  --> This file is 2.3MB.
Current Time is Wed Jan 22, 2020 20:22:08 for Done with Parsing Album Files For ModVal 82
Process [{0}] took 2.1 minutes.
Current Time is Wed Jan 22, 2020 20:22:08 for Parsing Album Files For ModVal 83
  Lo

  --> This file is 1.4MB.
Saving 6465 new artist IDs to /Volumes/Music/Discog/albums-db/metadata/84-ArtistAlbums.p
    Found 0   albums (7    ) in 3000/6642	/Volumes/Music/Discog/albums/96/309296
    Found 0   albums (8    ) in 3500/6642	/Volumes/Music/Discog/albums/96/3466096
    Found 0   albums (9    ) in 4000/6642	/Volumes/Music/Discog/albums/96/3890696
  --> This file is 3.8MB.
Current Time is Wed Jan 22, 2020 20:27:24 for Done with Parsing Album Files For ModVal 84
Process [{0}] took 3.2 minutes.
Current Time is Wed Jan 22, 2020 20:27:24 for Parsing Album Files For ModVal 85
  Looking for artist albums in /Volumes/Music/Discog/albums/85
    Found 0   albums (10   ) in 4500/6642	/Volumes/Music/Discog/albums/96/444496
    Found 0   albums (11   ) in 5000/6642	/Volumes/Music/Discog/albums/96/4998296
    Found 0   albums (12   ) in 5500/6642	/Volumes/Music/Discog/albums/96/5471396
    Found 0   albums (16   ) in 6000/6642	/Volumes/Music/Discog/albums/96/677296
    Found 0   albums (1

  Found 3756 artist directories
    Found 0   albums (0    ) in 0/3756	/Volumes/Music/Discog/albums/87/1000587
    Found 0   albums (0    ) in 100/3756	/Volumes/Music/Discog/albums/87/109287
  --> This file is 1.1MB.
Saving 5691 new artist IDs to /Volumes/Music/Discog/albums-db/metadata/98-ArtistAlbums.p
  --> This file is 2.0MB.
Current Time is Wed Jan 22, 2020 20:32:10 for Done with Parsing Album Files For ModVal 98
Process [{0}] took 2.0 minutes.
Current Time is Wed Jan 22, 2020 20:32:10 for Parsing Album Files For ModVal 99
  Looking for artist albums in /Volumes/Music/Discog/albums/99
    Found 0   albums (1    ) in 500/3756	/Volumes/Music/Discog/albums/87/1505987
    Found 0   albums (4    ) in 1000/3756	/Volumes/Music/Discog/albums/87/2070987
    Found 0   albums (11   ) in 1500/3756	/Volumes/Music/Discog/albums/87/2759487
    Found 0   albums (19   ) in 2000/3756	/Volumes/Music/Discog/albums/87/3544487
  Found 6690 artist directories
    Found 0   albums (0    ) in 0/6690	/Volu